In [5]:
using Revise, OrdinaryDiffEq, Kinetic, BenchmarkTools
using KitBase.JLD2, KitBase.ProgressMeter, KitBase.OffsetArrays
using KitML.Flux, KitML.DiffEqFlux, KitML.Optim

In [157]:
matter = "gas"
case = "shock"
space = "1d2f1v"
nSpecies = 1
interpOrder = 1
flux = "kfvs"
collision = "bgk"
limiter = "vanleer"
boundary = "fix"
cfl = 0.7
maxTime = 250.0
x0 = -25.0
x1 = 25.0
nx = 80
pMeshType = "uniform"
nxg = 1
umin = -10.0
umax = 10.0
nu = 72
nug = 0
vmin = -10.0
vmax = 10.0
nv = 36
nvg = 0
wmin = -10.0
wmax = 10.0
nw = 36
nwg = 0
vMeshType = "rectangle"
knudsen = 1.0
mach = 3.0
prandtl = 0.6666667
inK = 2
omega = 0.5
alphaRef = 1.0
omegaRef = 0.5
nm = 5
tLen = 3
nh = 12

12

In [158]:
γ = heat_capacity_ratio(inK, 1)
set = Setup(matter, case, space, flux, collision, nSpecies, interpOrder, limiter, boundary, cfl, maxTime)
pSpace = PSpace1D(x0, x1, nx, nxg)
μᵣ = ref_vhs_vis(knudsen, alphaRef, omegaRef)
gas = Gas(knudsen, mach, prandtl, inK, γ, omega, alphaRef, omegaRef, μᵣ)
vSpace = VSpace1D(umin, umax, nu, vMeshType)
vSpace2D = VSpace2D(vmin, vmax, nv, wmin, wmax, nw, vMeshType)
vSpace3D = VSpace3D(umin, umax, nu, vmin, vmax, nv, wmin, wmax, nw, vMeshType)
wL, primL, hL, bL, bcL, wR, primR, hR, bR, bcR = ib_rh(mach, γ, inK, vSpace.u)
ib = IB2F(wL, primL, hL, bL, bcL, wR, primR, hR, bR, bcR)
ks = SolverSet(set, pSpace, vSpace, gas, ib, pwd())

kn_bzm = hs_boltz_kn(ks.gas.μᵣ, 1.0)
sos = sound_speed(ks.ib.primR, γ)
tmax = (ks.vSpace.u1 + sos) / ks.pSpace.dx[1]
dt = Float32(ks.set.cfl / tmax)
tspan = (0.f0, dt)
tran = range(tspan[1], tspan[2], length = tLen)

0.0f0:0.018620167f0:0.037240334f0

In [159]:
phi, psi, phipsi = kernel_mode(
    nm,
    vSpace3D.u1,
    vSpace3D.v1,
    vSpace3D.w1,
    vSpace3D.du[1, 1, 1],
    vSpace3D.dv[1, 1, 1],
    vSpace3D.dw[1, 1, 1],
    vSpace3D.nu,
    vSpace3D.nv,
    vSpace3D.nw,
    alphaRef,
);

In [160]:
ctr = OffsetArray{ControlVolume1D2F}(undef, eachindex(ks.pSpace.x))
face = Array{Interface1D2F}(undef, ks.pSpace.nx + 1)
for i in eachindex(ctr)
    if i <= ks.pSpace.nx ÷ 2
        ctr[i] = ControlVolume1D2F(
            ks.pSpace.x[i],
            ks.pSpace.dx[i],
            Float32.(ks.ib.wL),
            Float32.(ks.ib.primL),
            Float32.(ks.ib.hL),
            Float32.(ks.ib.bL),
        )
    else
        ctr[i] = ControlVolume1D2F(
            ks.pSpace.x[i],
            ks.pSpace.dx[i],
            Float32.(ks.ib.wR),
            Float32.(ks.ib.primR),
            Float32.(ks.ib.hR),
            Float32.(ks.ib.bR),
        )
    end
end
for i = 1:ks.pSpace.nx+1
    face[i] = Interface1D2F(ks.ib.wL, ks.ib.hL)
end

In [162]:
sumRes = zeros(3)
sumAvg = zeros(3)

@showprogress for iter = 1:2000
    Kinetic.evolve!(ks, ctr, face, dt)
    Kinetic.update!(ks, ctr, face, dt, sumRes)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


In [163]:
X = Array{Float32}(undef, ks.vSpace.nu * 2, ks.pSpace.nx)
for i = 1:ks.pSpace.nx
    X[1:nu, i] .= ctr[i].h
    X[nu+1:end, i] .= ctr[i].b
end

In [164]:
function shakhov!(df, f, p, t)
    H, B, tau = p
    df[1:end÷2, :] .= (H .- f[1:end÷2, :]) ./ tau
    df[end÷2+1:end, :] .= (B .- f[end÷2+1:end, :]) ./ tau
end

H = Array{Float32}(undef, nu, size(X, 2))
B = Array{Float32}(undef, nu, size(X, 2))
SH = Array{Float32}(undef, nu, size(X, 2))
SB = Array{Float32}(undef, nu, size(X, 2))
τ = Array{Float32}(undef, 1, size(X, 2))
for i in axes(X, 2)
    H[:, i] .= maxwellian(ks.vSpace.u, ctr[i].prim)
    B[:, i] .= H[:, i] .* ks.gas.K ./ (2.0 .* ctr[i].prim[end])
    
    q = heat_flux(ctr[i].h, ctr[i].b, ctr[i].prim, ks.vSpace.u, ks.vSpace.weights)
    H1, B1 = shakhov(ks.vSpace.u, H[:,i], B[:,i], q, ctr[i].prim, ks.gas.Pr, ks.gas.K)
    SH[:,i] .= H[:,i] .+ H1
    SB[:,i] .= B[:,i] .+ B1
    
    τ[1, i] = vhs_collision_time(ctr[i].prim, ks.gas.μᵣ, ks.gas.ω)
end
P = [SH, SB, τ]
M = vcat(H, B)

prob = ODEProblem(shakhov!, X, tspan, P)
Y = solve(prob, Euler(), dt=dt) |> Array;

In [165]:
#--- universal differential equation ---#
model_univ = FastChain(
    #(x, p) -> zeros(eltype(x), axes(x)),
    FastDense(nu * 2, nu * 2 * nh, tanh),
    #FastDense(nu * 2 * nh, nu * 2 * nh, tanh),
    FastDense(nu * 2 * nh, nu * 2),
    #(x, p) -> sum(x[1:nu] .* ks.vSpace.weights),
)

p_model = initial_params(model_univ)
#p_model .= 0.f0

function dfdt(f, p, t)
    h = f[1:nu, :]
    b = f[nu+1:end, :]

    dh = (H .- h) ./ τ .+ model_univ(f .- M, p)[1:nu, :]
    db = (B .- b) ./ τ .+ model_univ(f .- M, p)[nu+1:end, :]

    df = vcat(dh, db)
end

prob_ube = ODEProblem(dfdt, X, tspan, p_model)

function loss(p)
    #sol_ube = solve(prob_ube, Midpoint(), u0=X, p=p, saveat=tran)
    sol_ube = solve(prob_ube, Euler(), u0=X, p=p, dt=dt)
    loss = sum(abs2, Array(sol_ube) .- Y)
    return loss
end

cb = function (p, l)
    display(l)
    return false
end

res = sci_train(loss, p_model, ADAM(), cb=Flux.throttle(cb, 1), maxiters=200)

0.0030288259f0

5.1577226f-5

1.3563861f-5

9.211794f-6

7.931367f-6

6.8329036f-6

5.829224f-6

u: 499536-element Vector{Float32}:
 -0.033344563
  0.04711396
 -0.06453871
 -0.07521937
  0.026386881
 -0.01088655
 -0.011478854
  0.023843408
 -0.044298142
 -0.03802728
 -0.008793371
 -0.044114206
 -0.021723539
  ⋮
  0.00036814067
  0.00017772302
 -2.1439204f-5
  0.0009703685
 -2.3407436f-5
 -0.0002911765
 -0.00018774779
  2.3935832f-5
  0.0003100039
 -0.0009955408
  0.0003907911
 -0.00050813286

In [168]:
@benchmark model_univ([ctr[1].h;ctr[1].b], res.u)

BenchmarkTools.Trial: 
  memory estimate:  4.78 MiB
  allocs estimate:  25
  --------------
  minimum time:     412.694 μs (0.00% GC)
  median time:      502.258 μs (0.00% GC)
  mean time:        619.987 μs (17.92% GC)
  maximum time:     5.845 ms (89.06% GC)
  --------------
  samples:          8008
  evals/sample:     1

In [184]:
function bench_ube(df, f, p, t)
    H = p[1:nu]
    B = p[nu+1:2*nu]
    τ = p[2*nu+1]
    p_nn = p[2*nu+2:end]

    #H,B,τ,p_nn = p
    
    h = f[1:nu]
    b = f[nu+1:end]

    dh = (H .- h) ./ τ .+ model_univ(f, p_nn)[1:nu]
    db = (B .- b) ./ τ .+ model_univ(f, p_nn)[nu+1:end]

    df[1:nu] .= dh
    df[nu+1:end] .= db 
end

bench_ube (generic function with 1 method)

In [185]:
prob_ube = ODEProblem(bench_ube, [ctr[1].h[:]; ctr[1].b[:]], tspan, [H[:, 1]; B[:, 1]; τ[1, 1]; res.u])

ODEProblem with uType Vector{Float32} and tType Float32. In-place: true
timespan: (0.0f0, 0.037240334f0)
u0: Float32[5.191017f-16, 2.4065318f-15, 1.0656098f-14, 4.5053408f-14, 1.8181008f-13, 6.9998147f-13, 2.569968f-12, 8.993069f-12, 2.9975133f-11, 9.5100705f-11  …  3.4674946f-10, 2.5110719f-11, 1.5603014f-12, 8.318861f-14, 3.80562f-15, 1.4938012f-16, 5.031142f-18, 1.4539391f-19, 3.6052223f-21, 7.670505f-23]

In [187]:
#@btime solve(prob_ube, Euler(), dt=tspan[2]/(tLen-1))
@btime solve(prob_ube, BS3());

  17.529 ms (1283 allocations: 171.96 MiB)


In [80]:
model_reduce = FastChain(FastDense(nu, nu * nh),
    FastDense(nu * nh, nu))

(::FastChain{Tuple{FastDense{typeof(identity), DiffEqFlux.var"#initial_params#73"{Vector{Float32}}}, FastDense{typeof(identity), DiffEqFlux.var"#initial_params#73"{Vector{Float32}}}}}) (generic function with 1 method)

In [81]:
function bench_reduce(df, f, p, t)
    H = p[1:nu]
    τ = p[nu+1]
    p_nn = p[nu+2:end]

    df .= (H .- f) ./ τ .+ model_reduce(f, p_nn)[1:nu]
end

bench_reduce (generic function with 1 method)

In [82]:
prob_reduce = ODEProblem(bench_reduce, ctr[1].h[:], tspan, [H[:, 1]; τ[1, 1]; res.u])

ODEProblem with uType OffsetVector{Float32, Vector{Float32}} and tType Float32. In-place: true
timespan: (0.0f0, 0.037240334f0)
u0: Float32[5.191017f-16, 2.4065318f-15, 1.0656098f-14, 4.5053408f-14, 1.8181008f-13, 6.9998147f-13, 2.569968f-12, 8.993069f-12, 2.9975133f-11, 9.5100705f-11  …  3.440243f-10, 2.4913365f-11, 1.5480382f-12, 8.2534796f-14, 3.77571f-15, 1.4820608f-16, 4.991599f-18, 1.4425116f-19, 3.5768857f-21, 7.610217f-23]

In [83]:
@benchmark solve(prob_reduce, Midpoint())

BenchmarkTools.Trial: 
  memory estimate:  184.91 MiB
  allocs estimate:  1319
  --------------
  minimum time:     16.054 ms (0.00% GC)
  median time:      20.894 ms (22.68% GC)
  mean time:        20.481 ms (20.81% GC)
  maximum time:     22.438 ms (0.00% GC)
  --------------
  samples:          244
  evals/sample:     1

In [169]:
@benchmark begin
    f_full = full_distribution(ctr[1].h, ctr[1].b, ks.vSpace.u, ks.vSpace.weights, vSpace3D.v, vSpace3D.w, ctr[1].prim, ks.gas.γ)
    boltzmann_fft(f_full, kn_bzm, nm, phi, psi, phipsi)
end

BenchmarkTools.Trial: 
  memory estimate:  214.42 MiB
  allocs estimate:  1839
  --------------
  minimum time:     66.137 ms (0.00% GC)
  median time:      71.821 ms (7.21% GC)
  mean time:        71.547 ms (6.83% GC)
  maximum time:     72.999 ms (7.07% GC)
  --------------
  samples:          70
  evals/sample:     1

In [177]:
function bench_fsm(df, f, p, t)
    h = f[1:nu]
    b = f[nu+1:end]
    
    f_full = full_distribution(h, b, ks.vSpace.u, ks.vSpace.weights, vSpace3D.v, vSpace3D.w, ctr[1].prim, ks.gas.γ)

    _df = boltzmann_fft(f_full, kn_bzm, nm, phi, psi, phipsi)
    
    dh, db = reduce_distribution(_df, vSpace3D.v, vSpace3D.w, vSpace2D.weights)
    df[1:nu] .= dh
    df[nu+1:end] .= db 
end

bench_fsm (generic function with 1 method)

In [178]:
prob_fsm = ODEProblem(bench_fsm, [ctr[1].h[:]; ctr[1].b[:]], tspan)

ODEProblem with uType Vector{Float32} and tType Float32. In-place: true
timespan: (0.0f0, 0.037240334f0)
u0: Float32[5.191017f-16, 2.4065318f-15, 1.0656098f-14, 4.5053408f-14, 1.8181008f-13, 6.9998147f-13, 2.569968f-12, 8.993069f-12, 2.9975133f-11, 9.5100705f-11  …  3.4674946f-10, 2.5110719f-11, 1.5603014f-12, 8.318861f-14, 3.80562f-15, 1.4938012f-16, 5.031142f-18, 1.4539391f-19, 3.6052223f-21, 7.670505f-23]

In [183]:
@btime solve(prob_fsm, Tsit5())
#@btime solve(prob_fsm, Euler(), dt=tspan[2]/(tLen-1));

  669.710 ms (20699 allocations: 1.92 GiB)
